<a href="https://colab.research.google.com/github/anshulsolanki/stock_analysis/blob/main/RS-index_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install dependency if needed:
!pip install yfinance matplotlib
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.5 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import matplotlib.pyplot as plt
import os

# Define the indices and their Yahoo Finance symbols
indices = {
    "Nifty 50": "^NSEI",
    "Bank Nifty": "^NSEBANK",
    "Nifty IT": "^CNXIT",
    "Nifty Pharma": "^CNXPHARMA",
    "Nifty FMCG": "^CNXFMCG",
    "Nifty Auto": "^CNXAUTO",
    "Nifty Metal": "^CNXMETAL"
}

# Fetch last 5 years of data
data = {}
for name, symbol in indices.items():
    df = yf.download(symbol, period="5y", interval="1d")
    data[name] = df
    print(f"{name} → {len(df)} rows fetched")

# Create directory if it doesn't exist
if not os.path.exists("nifty_data"):
    os.makedirs("nifty_data")

# Save
for name, df in data.items():
    df.to_pickle(f"nifty_data/{name.replace(' ', '_')}.pkl")

/tmp/ipython-input-4210132491.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4210132491.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")


Nifty 50 → 1237 rows fetched


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4210132491.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")


Bank Nifty → 1231 rows fetched


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4210132491.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")


...

Nifty FMCG → 1222 rows fetched


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-4210132491.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="5y", interval="1d")


Nifty Auto → 1222 rows fetched


[*********************100%***********************]  1 of 1 completed

Nifty Metal → 1222 rows fetched


In [20]:
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta

# Assume `data` is your 5-year DataFrame dictionary
# data = {"Nifty 50": df_nifty50, "Bank Nifty": df_banknifty, ...}

sectors = ["Bank Nifty", "Nifty IT", "Nifty Pharma", "Nifty FMCG", "Nifty Auto", "Nifty Metal"]
colors = {
    "Bank Nifty": "blue",
    "Nifty IT": "green",
    "Nifty Pharma": "purple",
    "Nifty FMCG": "orange",
    "Nifty Auto": "red",
    "Nifty Metal": "brown"
}

time_frames = ["1mo", "3mo", "6mo", "1y", "2y", "5y"]

def get_start_date(df, tf):
    today = df.index.max()
    if tf.endswith("y"):
        years = int(tf[:-1])
        return today - pd.DateOffset(years=years)
    elif tf.endswith("mo"):
        months = int(tf[:-2])
        return today - pd.DateOffset(months=months)
    elif tf.endswith("d"):
        days = int(tf[:-1])
        return today - timedelta(days=days)
    else:
        raise ValueError("Invalid time frame")

# Initialize figure with empty traces
fig = go.Figure()
for sector in sectors:
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines", name=sector, line=dict(color=colors.get(sector))))

# Add horizontal line RS=1
fig.add_trace(go.Scatter(x=[], y=[], mode="lines", line=dict(color="black", dash="dash"), name="RS=1"))

# Create dropdown buttons
buttons = []
for tf in time_frames:
    x_data = []
    y_data = []
    for sector in sectors:
        combined = pd.concat([data[sector]["Close"], data["Nifty 50"]["Close"]], axis=1, join='inner')
        combined.columns = ["Sector", "Nifty50"]
        start_date = get_start_date(combined, tf)
        combined_slice = combined[combined.index >= start_date]

        # Compute normalized RS relative to first point of time frame
        rs = combined_slice["Sector"] / combined_slice["Nifty50"]
        rs = rs / rs.iloc[0]  # normalization at start of time frame

        x_data.append(rs.index)
        y_data.append(rs.values)

    # Horizontal RS=1 line
    x_rs1 = x_data[0]
    y_rs1 = [1]*len(x_rs1)
    x_data.append(x_rs1)
    y_data.append(y_rs1)

    buttons.append(dict(
        label=tf,
        method="update",
        args=[{"x": x_data, "y": y_data},
              {"title": f"Relative Strength vs Nifty 50 ({tf})"}]
    ))

fig.update_layout(
    updatemenus=[dict(active=3, buttons=buttons, x=0.05, y=1.15, xanchor="left", yanchor="top")],
    title="Relative Strength vs Nifty 50",
    xaxis_title="Date",
    yaxis_title="Normalized Sector / Nifty50 Ratio",
    legend_title="Sectors"
)

fig.show()
